In [1]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

In [2]:

bob           = sy.VirtualWorker(hook, id='bob')
alice         = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [8]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])


W0715 09:22:04.133381 4321358720 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


W0715 09:22:04.137709 4321358720 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior


W0715 09:22:04.139983 4321358720 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


W0715 09:22:04.141834 4321358720 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior


W0715 09:22:04.143430 4321358720 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


W0715 09:22:04.146822 4321358720 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:40>

In [16]:
# Dataset
bob.clear_objects()
alice.clear_objects()
secure_worker.clear_objects()

data   = th.tensor([[0.,0], [0,1], [1,0], [1,1]], requires_grad=True)
target = th.tensor([[0.], [0], [1], [1]], requires_grad=True)

bobs_data   = data[:2].send(bob)
bobs_target = target[:2].send(bob)

alices_data   = data[2:].send(alice)
alices_target = target[2:].send(alice)


In [17]:
def model_learning(obj_model, obj_data, obj_target):
    obj_opt  = optim.SGD(params=bobs_model.parameters(),   lr=0.1)
    
    obj_opt.zero_grad()
    obj_pred = obj_model(obj_data)
    
    obj_loss = ((obj_pred - obj_target)**2).sum()
    obj_loss.backward()
    
    obj_opt.step()
    
    return obj_loss.get().data

In [19]:

model = nn.Linear(2,1)
print('weight before:', model.weight)
print('bias before:', model.bias)

model.share(bob, alice, secure_worker)
print('secure_worker._objects:', secure_worker._objects)
 
for round_iter in range(10):
    bob_loss   = th.FloatTensor
    alice_loss = th.FloatTensor

    for i in range(10):
        bob_loss   = model_learning(model, bobs_data,   bobs_target)
        alice_loss = model_learning(model, alices_data, alices_target)
    else:
        print('Bob loss:', str(bob_loss), 'Alice loss:', str(alice_loss))

model = model.get().float_prec()

print('weight after:', model.weight)
print('bias after:', model.bias)

weight before: Parameter containing:
tensor([[-0.1007, -0.3046]], requires_grad=True)
bias before: Parameter containing:
tensor([0.6163], requires_grad=True)
secure_worker._objects: {90788696555: tensor([[-1304250385693443513, -3575292756196142439]]), 60140161125: tensor([-1560545915733068229]), 74488906432: tensor([[-3598892492180588465, -2582122572278142637]]), 50734585486: tensor([-4281181727780343679])}


KeyError: 'Object "97825388076" not found on worker!!!You just tried to interact with an object ID:97825388076 on <VirtualWorker id:secure_worker #objects:4> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'